# User study questionnaire  
Mock and plots

Reference: KMunicate - https://bmjopen.bmj.com/content/9/9/e030215


In [ ]:
# TODO: generate 7 features: X0, XA, XA, XAY, XAY, XY, XY


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import seaborn.objects as so
import matplotlib as mpl
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.special import expit

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from causallib.estimation import IPW


Generate data

In [5]:
def generate_data(n=1000, seed=0):
    rng = np.random.default_rng(seed)

    # X0, XA, XA, XAY, XAY, XY, XY
    X = rng.normal(0, 0.5, size=(n, 7))
    a_beta = np.array([0, 1, 1, 1, 1, 0, 0])
    a_logit = X @ a_beta
    a_prop = expit(a_logit)
    a = rng.binomial(1, a_prop)

    y_beta = np.array([0, 0, 0, -1, -1, -1, -1])
    effect = 1
    y = X @ y_beta + a * effect + rng.normal(0, 1, size=n)
    
    X = pd.DataFrame(X, columns=["x0", "xa1", "xa2", "xay1", "xay2", "xy1", "xy2"])
    a = pd.Series(a, name="a")
    y = pd.Series(y, name="y")
    return X, a, y

In [6]:
X, a, y = generate_data()
X.join(a).join(y)

,x0,xa1,xa2,xay1,xay2,xy1,xy2,a,y
0,0.062865,-0.066052,0.320211,0.052450,-0.267835,0.180798,0.652000,0,-2.993001
1,0.473540,-0.351868,-0.632711,-0.311637,0.020663,-1.162515,-0.109396,0,0.831974
2,-0.622955,-0.366134,-0.272129,-0.158150,0.205815,0.521257,-0.064267,0,0.748810
3,0.683232,-0.332597,0.175755,0.451735,0.047006,-0.371750,-0.460863,1,0.078306
4,-0.228863,0.110098,-0.504809,-0.104588,-0.079613,0.270423,0.107330,0,2.070282
...,...,...,...,...,...,...,...,...,...
995,1.006685,0.429447,-0.938928,0.291366,-0.522681,0.353278,0.725023,0,-2.776098
996,-0.055806,-0.888242,-0.146091,-0.537013,-0.044161,-0.226326,0.754823,0,0.474894
997,-1.140338,-0.340710,-0.209949,-0.349892,-0.063579,0.189120,0.923519,0,0.546473
998,-0.010102,-0.960817,-0.080658,-0.212401,0.475722,-0.128377,0.629057,1,0.614852


(weighted) ASMD:

In [7]:
ipw = IPW(LogisticRegression(penalty="none"))
ipw.fit(X, a)
w = ipw.compute_weights(X, a)

In [8]:
def calculate_asmd(X, a, w=None):
    eps = np.finfo(X.dtypes.iloc[0]).resolution  # .eps
    if w is None:
        w = pd.Series(1, index=a.index)
    
    is_treated = a == 1
    x1 = sm.stats.DescrStatsW(X.loc[is_treated], weights=w.loc[is_treated])
    x0 = sm.stats.DescrStatsW(X.loc[~is_treated], weights=w.loc[~is_treated])

    x1_mean = pd.Series(x1.mean, index=X.columns)
    x0_mean = pd.Series(x0.mean, index=X.columns)
    x1_var = pd.Series(x1.var, index=X.columns)
    x0_var = pd.Series(x0.var, index=X.columns)

    smds = (x1_mean - x0_mean) / np.sqrt(x0_var + x1_var + eps)
    asmds = smds.abs()
    asmds.name = "asmd"
    return asmds

In [9]:
asmds = pd.concat({
    "weighted": calculate_asmd(X, a, w),
    "unweighted": calculate_asmd(X, a),
}, names=["adjustment", "covariate"])
asmds

adjustment  covariate
weighted    x0           0.006971
            xa1          0.006630
            xa2          0.001237
            xay1         0.001711
            xay2         0.009370
            xy1          0.010796
            xy2          0.011687
unweighted  x0           0.045328
            xa1          0.288934
            xa2          0.269385
            xay1         0.308031
            xay2         0.370553
            xy1          0.036131
            xy2          0.024381
Name: asmd, dtype: float64

Outcome importance

In [10]:
def leave_one_out_importance(estimator, X, a, y):
    results = []

    for col in ["full"] + X.columns.tolist():
        curX = X.drop(columns=col, errors="ignore")
        curXa = curX.join(a)
        estimator.fit(curXa, y)
        y_pred = estimator.predict(curXa)
        result = {
            "covariate": col,
            "r2": r2_score(y, y_pred),
            "mse": mean_squared_error(y, y_pred),
            "mae": mean_absolute_error(y, y_pred),
        }
        results.append(result)
    results = pd.DataFrame(results)
    return results

def relative_explained_variation(estimator, X, a, y, metric="mse"):
    """Harrell: https://www.fharrell.com/post/addvalue/"""
    importance = leave_one_out_importance(estimator, X, a, y)
    importance = importance.set_index("covariate")
    importance = importance / importance.loc["full"]
    importance = importance.drop(index="full")
    # importance = importance[metric]
    return importance

def decrease_in_explain_variation(estimator, X, a, y, metric="mse"):
    """https://stackoverflow.com/q/31343563"""
    importance = leave_one_out_importance(estimator, X, a, y)
    importance = importance.set_index("covariate")
    importance = (importance.loc["full"]-importance) / importance.loc["full"]
    importance = importance.drop(index="full")
    # importance = importance[metric]
    importance = importance.abs()
    return importance


In [11]:
# i = leave_one_out_importance(LinearRegression(), X, a, y)
# i = i.set_index("covariate")
# i
# relative_explained_variation(LinearRegression(), X, a, y)
feature_importance = decrease_in_explain_variation(LinearRegression(), X, a, y)
feature_importance

,r2,mse,mae
covariate,,,
x0,0.000679,0.000640,0.000206
xa1,0.000014,0.000013,0.000081
xa2,0.000094,0.000089,0.000216
xay1,0.240087,0.226260,0.082807
xay2,0.209418,0.197358,0.097387
xy1,0.245376,0.231245,0.104534
xy2,0.260047,0.245071,0.112926


In [12]:
outcome_metric = "mse"

Outcome-informed ASMD score

In [24]:
ouiasmd = asmds.xs("unweighted", level="adjustment")
ouiasmd = ouiasmd * feature_importance[outcome_metric]
ouiasmd = asmds.reset_index().merge(
    ouiasmd.to_frame("ouiasmd"),
    on="covariate",
    how="left",
)
ouiasmd

,adjustment,covariate,asmd,ouiasmd
0,weighted,x0,0.006971,0.000029
1,weighted,xa1,0.006630,0.000004
2,weighted,xa2,0.001237,0.000024
3,weighted,xay1,0.001711,0.069695
4,weighted,xay2,0.009370,0.073131
5,weighted,xy1,0.010796,0.008355
6,weighted,xy2,0.011687,0.005975
7,unweighted,x0,0.045328,0.000029
8,unweighted,xa1,0.288934,0.000004
9,unweighted,xa2,0.269385,0.000024


In [ ]:
# TODO: generate results

## Main part
 * how much do you agree with the problem (Setting/setup)
 * how much do you agree with the need for a solution
 * how much did the visualization improve your interpretation. 
 [also allow for additional free-text comments on each]

### Participant details

* Principle role / background (school of causality)
* experience in causality years
* familiarity with Matching / IPW
* experience in reading/interpreting Love plots [years]   ?   # really break it down to reading/generating?
* experience in generating Love plots [years] ?
* continent of primary work


## Ablation
Compare against standard Love plot:
 * size
 * opacity
 * order
 * (size, opacity)
 * (size, opacity, order)
 [score each 0-10] [‘Less useful’, ‘Equal/no preference’, ‘A bit more useful’, ‘Somewhat more useful’ and ‘Much more useful’]
 [Then rank]
 [free text comments]